In [1]:
from pygsti.modelpacks import smq1Q_XY, smq1Q_ZN
from pygsti.tools.leakage import leaky_qubit_model_from_pspec, construct_leakage_report
from pygsti.data import simulate_data
from pygsti.protocols import StandardGST, ProtocolData

## GST: modeling a leaky qubit as a qutrit

This short notebook shows how (data from) an experiment design for a two-level system can be used to fit a three-level sytem model, and how to generate a special report to provide insights for these models. The report includes special gate error metrics that reflect the distinguished role of the first two levels in the three-level system.

In [2]:
mp = smq1Q_XY
ed = mp.create_gst_experiment_design(max_max_length=32)
tm3 = leaky_qubit_model_from_pspec(mp.processor_spec(), basis='l2p1')
# ^ We could use basis = 'gm' instead of 'l2p1'. We prefer 'l2p1'
#   because it makes process matrices easier to interpret in leakage
#   modeling.
ds = simulate_data(tm3, ed.all_circuits_needing_data, num_samples=1000, seed=1997)
gst = StandardGST( modes=('CPTPLND',), target_model=tm3, verbosity=2)
pd = ProtocolData(ed, ds)
res = gst.run(pd)

-- Std Practice:  Iter 1 of 1  (CPTPLND) --: 


/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/modelmembers/povms/__init__.py:615: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  errgen_vec = _np.linalg.lstsq(phys_directions, soln.x)[0]
/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/objectivefns/objectivefns.py:4471: RuntimeWarning: divide by zero encountered in divide
  p5over_lsvec = 0.5/lsvec


  --- Iterative GST: [##################################################] 100.0%  568 circuits ---


In [3]:
report_dir = 'example_files/leakage-report-automagic'
report_object, updated_res = construct_leakage_report(res, title='easy leakage analysis!')
# ^ Each estimate in updated_res has a new gauge-optimized model.
#   The gauge optimization was done to reflect how our target gates
#   are only _really_ defined on the first two levels of our
#   three-level system.
#   
report_object.write_html(report_dir)

Running idle tomography
Computing switchable properties


/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/models/model.py:143: UserWarning:

Model.num_modeltest_params could not obtain number of *non-gauge* parameters - using total instead

/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/report/plothelpers.py:367: UserWarning:

Max-model params (56) <= model params (87)!  Using k == 1.

/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/forwardsims/mapforwardsim.py:731: UserWarning:

Generating dense process matrix representations of circuits or gates 
can be inefficient and should be avoided for the purposes of forward 
simulation/calculation of circuit outcome probability distributions 
when using the MapForwardSimulator.

